<a href="https://colab.research.google.com/github/giangkarry/Woodblock_image_super_resolution/blob/main/X%E1%BB%AD%20l%C3%BD%20d%E1%BB%AF%20li%E1%BB%87u.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from PIL import Image, ImageOps, ImageFilter
import matplotlib.pyplot as plt

**Tính độ phân giải trung bình**

In [ ]:
data_dir = '/content/drive/MyDrive/6TDATA'

# Tìm trung bình chiều rộng, chiều cao
def findAvg(data_dir):
  widths = []
  heights = []  
  for filename in os.listdir(data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
      image = Image.open(os.path.join(data_dir, filename))
      widths.append(image.size[0])
      heights.append(image.size[1])
  avg_width = round(sum(widths)/len(widths))
  avg_height = round(sum(heights)/len(heights))
  return avg_width, avg_height

avg_width, avg_height = findAvg(data_dir)
print(f'average width: {avg_width}, average height: {avg_height}')



average width: 140, average height: 133


**Padding => width = height sau đó resize (128,128) với phép nội suy Bicubic**

In [ ]:

def pad (image):
  width, height = image.size
  diff = abs(width-height)

  if width > height:
    border = (0, diff//2, 0, diff-diff//2) # (trái, trên, phải, dưới)
    pad_image = ImageOps.expand(image, border=border, fill=(0, 0, 0))
    for y in range(height+diff): # height+diff = width 
      for x in range(width):
        if y < (diff//2):
          pixel_value = pad_image.getpixel((x,diff//2))
          pad_image.putpixel((x,y), pixel_value)
        if y >= (height+diff//2 ):
          pixel_value = pad_image.getpixel((x,height+diff//2-1))
          pad_image.putpixel((x,y), pixel_value)
  else:
    border = (diff//2, 0, diff-diff//2,0)
    pad_image = ImageOps.expand(image, border=border, fill=(0, 0, 0))
    for y in range(height):
      for x in range(width+diff):
        if x < diff//2:
          pixel_value = pad_image.getpixel((diff//2, y))
          pad_image.putpixel((x,y), pixel_value)
        if x >= (width+diff//2):
          pixel_value = pad_image.getpixel((width+diff//2-1, y))
          pad_image.putpixel((x,y), pixel_value)
  return pad_image


def resize_image(datadir, savedir, new_width, new_height):
  for filename in os.listdir(data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
      image = Image.open(os.path.join(data_dir, filename))
      pad_image = pad(image)
      final_image = pad_image.resize((new_width, new_height), resample=Image.BICUBIC) # resize(128,128) với phương pháp nội suy Bicubic
      final_image.save(savedir+filename)

savedir='/content/drive/MyDrive/Data/6TResize/HR/'
resize_image(data_dir, savedir,128, 128)



**Đổi tên ảnh**

In [ ]:

hr_dir = '/content/drive/MyDrive/Data/6TResize/HR'

files = [file for file in os.listdir(hr_dir) if file.endswith('.png')]

for i, file in enumerate(files):
  img = Image.open(os.path.join(hr_dir, file))
  new_name = f'{i+1}.png'
  img.save(os.path.join('/content/drive/MyDrive/Data/6TResize/TextIMG', new_name))


**Chuyển sang ảnh RGB**

In [ ]:

data_dir = '/content/drive/MyDrive/Data/6TResize/TextIMG'
rgb_dir = '/content/drive/MyDrive/Data/6TResize/RGB'

def rgba_to_rgb(data_dir, rgb_dir):
  for filename in os.listdir(data_dir):
    if filename.endswith('.png'):
      img = Image.open(os.path.join(data_dir, filename))
      rgb = img.convert('RGB')
      alpha = img.getchannel('A')
      rgb_img = Image.merge('RGB', (rgb.getchannel(0), rgb.getchannel(1), rgb.getchannel(2)))
      rgb_img.save(os.path.join(rgb_dir, filename))

rgba_to_rgb(data_dir, rgb_dir)
print(len(os.listdir(rgb_dir)))

6000


**Chia dữ liệu => train, test, val theo tỷ lệ 8:1:1**

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

hr_dir = '/content/drive/MyDrive/Data/6TResize/RGB'
train_dir = '/content/drive/MyDrive/Data/6TResize/SRDataset/train/high'
val_dir = '/content/drive/MyDrive/Data/6TResize/SRDataset/validation/high'
test_dir = '/content/drive/MyDrive/Data/6TResize/SRDataset/test/high'

#Tạo folder

if not os.path.exists(train_dir):
  os.makedirs(train_dir)
if not os.path.exists(val_dir):
  os.makedirs(val_dir)
if not os.path.exists(test_dir):
  os.makedirs(test_dir)

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

files = os.listdir(hr_dir)

# chia tập dữ liệu thành 2 phần train và test_val

train_files, test_val_files = train_test_split(files, test_size=(test_ratio+val_ratio), random_state=42)

# Chia dữ liệu test_val thành 2 phần test và val
test_files, val_files = train_test_split(test_val_files, test_size=((test_ratio)/(test_ratio+val_ratio)), random_state=42)

for file in train_files:
  shutil.copy(os.path.join(hr_dir, file), os.path.join(train_dir, file))

for file in val_files:
  shutil.copy(os.path.join(hr_dir, file), os.path.join(val_dir, file))

for file in test_files:
  shutil.copy(os.path.join(hr_dir, file), os.path.join(test_dir, file))



**Tạo hình ảnh có độ phân giải thấp**

In [ ]:
def lr_images_generation(hr_dir, lr_dir, upscale_factor):
  for file in os.listdir(hr_dir):
    if file.endswith('.jpg') or file.endswith('.png'):
      image = Image.open(os.path.join(hr_dir, file))
      width, height = image.size
      img_lr = image.resize((width//upscale_factor, height//upscale_factor), resample=Image.BICUBIC)
      img_lr.save(os.path.join(lr_dir, file))

hr_train = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/train/high'
hr_val = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/validation/high'
hr_test = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/test/high'

lr_train = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/train/low'
lr_val = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/validation/low'
lr_test = '/content/drive/MyDrive/Data/Resize/SRDatasetBlur/test/low'

lr_train_data = lr_images_generation(hr_train, lr_train, 4)
lr_val_data = lr_images_generation(hr_val, lr_val, 4)
lr_test_data = lr_images_generation(hr_test, lr_test, 4)

**Upscale image  x4 với bicubic interpolation**

In [ ]:
def hr_lr_images_copy(root_dir, hr_dir, lr_dir, upscale_factor):
  for file_name in os.listdir(hr_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
      hr_img = Image.open(os.path.join(hr_dir, file_name))
      lr_img = Image.open(os.path.join(lr_dir, file_name))
      width, height = lr_img.size
      bicubic_img = lr_img.resize((width*upscale_factor, height*upscale_factor), resample=Image.BICUBIC)
      hr_img.save(os.path.join(root_dir,'high',file_name))
      bicubic_img.save(os.path.join(root_dir,'low',file_name))
root_dir_train = "/content/drive/MyDrive/Data/6TResize/BicubicDataset/validation"
hr_dir_train ="/content/drive/MyDrive/Data/6TResize/SRDataset/validation/high"
lr_dir_train="/content/drive/MyDrive/Data/6TResize/SRDataset/validation/low"

hr_lr_images_copy(root_dir_train, hr_dir_train, lr_dir_train,4 )